# Image plates of duckweed in growth assay experiment

Ensure the following before running anything in this notebook:
 - Plates with duckweed are loaded in the correct positions on the bedplate
 - The machine is switched on, connected to the computer and all axes have been homed. 
 - The camera tool is accessible and in position 1 on the Jubilee
 
 Once those steps are complete run the cells in this notebook in order

In [8]:
#Importing python libraries downloaded from the internet and those saved locally
import serial 
from serial.tools import list_ports
import yaml
import matplotlib
matplotlib.use('TkAgg') #This is a specific back end of matplotlib 
from matplotlib import pyplot as plt
import numpy as np
import os
import cv2 as cv2
import time
import pandas as pd
import glob
import json
import datetime
from datetime import date
from utils.MachineUtils import * #Local library
from utils.CameraUtils import * #Local library. #Note that this library requirs a package that can only be installed on raspbery pi. 
import utils.Plate_positions as pp #Local library

import ipywidgets as widgets
# from Ipython.display import display, Image
import threading

In [10]:
ports = serial.tools.list_ports.comports()
ser = serial.Serial('/dev/ttyACM0', 115200) #Port address and Baudrate (info transfer rate)
port = '/dev/ttyACM0'
m = MachineCommunication(port)
ser.write(b'G90\n') #Command to make sure the Jubilee is using absolute as opposed to relative positioning
m.toolChange(1)

### 1. Define output directory

In [17]:
#Define directory to save image data for this experiment
data_dir = '/home/pi/Documents/Data'
expt_dir = data_dir + '/TestDriveExpt_1_08032022'
expt_id = "TestDriveExpt_1_08032022"
isExist = os.path.exists(expt_dir)
if not isExist:
  # Create a new directory because it does not exist 
    os.makedirs(expt_dir)
    print("The new directory is created!")
print("Image files will be saved to:")
print(expt_dir)


The new directory is created!
Image files will be saved to:
/home/pi/Documents/Data/TestDriveExpt_1_08032022


### 2. Option 1 - Read file to define plates to be imaged

In [26]:

#Automated option: Define experiment set up file and read from that which plates are to be imaged. 

expt_setup_parent_dir = "/home/pi/duckbot/notebooks/TestExptDir/" # Default uses current working directory but you can replace with your own choice. 
expt_setup_dir = "TestDriveExpt_1_08032022" # Name of the folder to hold experiment data and metadata including the setup file
expt_setup_file_name = "TestDriveExpt_1_08032022.json" #Name for the experiment setup file (Metadata)

exp_setup_file_path = os.path.join(expt_setup_parent_dir, expt_setup_dir)
print(exp_setup_file_path)

/home/pi/duckbot/notebooks/TestExptDir/TestDriveExpt_1_08032022


In [21]:
def move_across_platewells(well_coord_list_of_dicts):
    for i, well in enumerate(well_coord_list_of_dicts):
        m.moveTo(x=well["x"], y=well["y"], z=10)
        ser.write(b'G4 P1000') 

#TODO make sure it only images wells with duckweed in them. Not crucial since empty wells will be ignored 
# in data analysis but still its a waste of time and disk space. 
def image_plates(plates_to_image):
    reference = pp.whole_bedplate_positions
    for p in plates_to_image:
        print(p)
        for plate in reference:
            if int(p) == int(plate['plate']):
                well_coords = plate['well_dict']
                for i, well in enumerate(well_coords):
                    wellid = well['well_id']
                    print(f'move to well {wellid}')
                    m.moveTo(x=well["x"], y=well["y"], z=10)
                    m.dwell(500) #dwell .75 seconds
                    print('get frame')
                    f = getFrame()
                    print('write image to file')
                    cv2.imwrite(f'{expt_dir}/{expt_id}_plate{p}_well{well["well_id"]}_{date.today()}.jpg', f)
                    time.sleep(0.25)
                    
def move_to_first_well(plates_to_image):
    first_plate = plates_to_image[0]
    well = pp.fetch_well_position(first_plate,"A1")
    m.moveTo(x=well["x"], y=well["y"], z=10)

In [27]:
os.chdir(exp_setup_file_path)

with open(expt_setup_file_name) as datafile:
    expt_data = json.load(datafile)

# Turn samples list into a dataframe
sample_data = expt_data["sample_info"]
df = pd.DataFrame(sample_data)

unique_plates = list(df.Plate.unique())

def pull_last_number(n):
    return n[-1:]

plates_to_image = list(map(pull_last_number, unique_plates))
print(plates_to_image)


['1', '2']


### 2. Option 2 - Manually define plates to be imaged

In [ ]:
# Manual option: Which plates do you want to image (numbers correspond to positions on bedplate. Options: 1,2,3,4,5)
plates_to_image = [1]

### 3. Focus camera
Run the cells below. It will move the camera over the first well of the first plate and then pop up a live camera window. Manually adjust the focus to the desired point by twisting the housing of the camera lens. Then hit the escape key to close the window

In [14]:
#Move camera over first well to be imaged
move_to_first_well(plates_to_image)

In [ ]:
#Drop bed down and add plates

In [15]:
#Open a live video window
cap = cv2.VideoCapture(0) #Note that the index corresponding to your camera may not be zero but this is the most common default

while True:
    ret, frame = cap.read()
    cv2.imshow('Input', frame)
    c = cv2.waitKey(1)
    if c ==27: #27 is the built in code for ESC so press escape to close the window. 
        break 
        
cap.release()
cv2.destroyAllWindows()

### 3. Capture and save images

In [28]:
image_plates(plates_to_image)

1
move to well A1
get frame
CHANGE
... camera connection established
lets read in yml
got yml, lets undistort
write image to file
move to well A2
get frame
CHANGE
... camera connection established
lets read in yml
got yml, lets undistort
write image to file
move to well A3
get frame
CHANGE
... camera connection established
lets read in yml
got yml, lets undistort
write image to file
move to well A4
get frame
CHANGE
... camera connection established
lets read in yml
got yml, lets undistort
write image to file
move to well A5
get frame
CHANGE
... camera connection established
lets read in yml
got yml, lets undistort
write image to file
move to well A6
get frame
CHANGE
... camera connection established
lets read in yml
got yml, lets undistort
write image to file
move to well B1
get frame
CHANGE
... camera connection established
lets read in yml
got yml, lets undistort
write image to file
move to well B2
get frame
CHANGE
... camera connection established
lets read in yml
got yml, lets undi

KeyboardInterrupt: 